In [1]:
# Importação das bibliotecas utilizadas
import numpy as np 
import pandas as pd
import plotly.express as px

# Para esta análise foram consideradas as bases de produto e transações, 
# pois o esperado é a indicação de produtos ou categorias que gerem maior resultado
product = pd.read_csv('../input/archives/product.csv')
transaction = pd.read_csv('../input/archives/transaction_data.csv')

In [2]:
product.head()

,PRODUCT_ID,MANUFACTURER,DEPARTMENT,BRAND,COMMODITY_DESC,SUB_COMMODITY_DESC,CURR_SIZE_OF_PRODUCT
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,
2,26093,69,PASTRY,Private,BREAD,BREAD:ITALIAN/FRENCH,
3,26190,69,GROCERY,Private,FRUIT - SHELF STABLE,APPLE SAUCE,50 OZ
4,26355,69,GROCERY,Private,COOKIES/CONES,SPECIALTY COOKIES,14 OZ


In [3]:
# O dataset de produto possui 92 mil linhas e 7 colunas
product.shape

(92353, 7)

In [4]:
transaction.head()

,household_key,BASKET_ID,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,TRANS_TIME,WEEK_NO,COUPON_DISC,COUPON_MATCH_DISC
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [5]:
# O dataset de transações possui 106 mil linhas e 12 colunas
transaction.shape

(106095, 12)

In [6]:
# Agregando os datasets para visualizar a tendência dos produtos
data_base = transaction[['household_key', 'DAY', 'PRODUCT_ID', 'QUANTITY', 'SALES_VALUE']] \
    .merge(product[['PRODUCT_ID', 'COMMODITY_DESC', 'SUB_COMMODITY_DESC']], on='PRODUCT_ID')

data_base.head()

,household_key,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,COMMODITY_DESC,SUB_COMMODITY_DESC
0,2375,1,1004906,1,1.39,POTATOES,POTATOES RUSSET (BULK&BAG)
1,1929,4,1004906,1,1.39,POTATOES,POTATOES RUSSET (BULK&BAG)
2,568,4,1004906,1,1.39,POTATOES,POTATOES RUSSET (BULK&BAG)
3,887,5,1004906,1,1.39,POTATOES,POTATOES RUSSET (BULK&BAG)
4,175,7,1004906,1,1.99,POTATOES,POTATOES RUSSET (BULK&BAG)


In [7]:
# Excluindo linhas vazias da coluna COMMODITY_DESC, e a categoria COUPON/MISC ITEMS,
# pois a mesma possui uma grande variação de produtos genéricos
data_base = data_base [~data_base ['COMMODITY_DESC'].isin(['',' ','COUPON/MISC ITEMS'])] 

daily_sales = data_base.groupby(['COMMODITY_DESC', 'DAY']).agg(
    {'SALES_VALUE': 'sum', 'QUANTITY': 'sum', 'household_key': pd.Series.nunique}) \
    .reset_index()

daily_sales.head()

,COMMODITY_DESC,DAY,SALES_VALUE,QUANTITY,household_key
0,(CORP USE ONLY),13,0.95,1,1
1,(CORP USE ONLY),48,1.84,2,1
2,(CORP USE ONLY),51,1.06,1,1
3,(CORP USE ONLY),68,2.34,1,1
4,(CORP USE ONLY),71,4.00,4,1


In [8]:
# Dataframe para visualizar os valores totais de venda por categoria
sales = daily_sales[['COMMODITY_DESC', 'SALES_VALUE']] \
    .groupby(['COMMODITY_DESC']).sum().reset_index()

# Para melhor visualização nos gráficos serão consideradas as 10 primeiras categorias
sales = sales[sales['COMMODITY_DESC'] != ''].sort_values(by='SALES_VALUE', ascending=False)[:10] \
    .sort_values(by='SALES_VALUE')


fig = px.bar(x=sales.SALES_VALUE,
             y=sales.COMMODITY_DESC,
             labels={
                 'y': 'Categoria',
                 'x': 'Valor em vendas'
             },
             title='Valor de vendas por Categoria',
             template='simple_white')
fig.show()

A Categoria BEEF é a principal em termos de venda, seguida de SOFT DRINKS.

In [9]:
# Dataframe para visualizar as quantidades de vendas por categoria
quantity = daily_sales[['COMMODITY_DESC', 'QUANTITY']] \
    .groupby(['COMMODITY_DESC']).sum().reset_index()

# Para melhor visualização nos gráficos serão consideradas as 10 primeiras categorias
quantity = quantity[quantity['COMMODITY_DESC'] != ''].sort_values(by='QUANTITY', ascending=False)[:10] \
    .sort_values(by='QUANTITY')


fig = px.bar(x=quantity.QUANTITY,
             y=quantity.COMMODITY_DESC,
             labels={
                 'y': 'Categoria',
                 'x': 'Quantidade'
             },
             title='Quantidades por Categoria',
             template='simple_white')
fig.show()

Analisando as quantidades, BEEF já não aparece nas primeiras posições, enquanto SOFT DRINK continua sendo a principal categoria de consumo.

In [10]:
list_sales = sales['COMMODITY_DESC']

top10_sales = daily_sales[daily_sales['COMMODITY_DESC'] \
                               .isin(list_sales)] \
                                [['COMMODITY_DESC', 'SALES_VALUE', 'DAY']] \
                               .groupby(['COMMODITY_DESC','DAY']).sum().reset_index()




fig = px.box(sales, 
              x = top10_sales.COMMODITY_DESC, 
              y = top10_sales.SALES_VALUE,
             color = top10_sales.COMMODITY_DESC,
             title = 'Distribuição de Vendas por Categoria',
             template = 'simple_white')
fig.show()

A categoria BEEF tem a maior variação, com vários outliers puxando o valor médio, SOFT DRINKS também apresenta uma grande variação.

In [11]:
sales_agg = data_base.groupby(['COMMODITY_DESC']) \
            .agg({'SALES_VALUE':'sum', 'QUANTITY':'sum', 'household_key':pd.Series.nunique}) \
            .rename(columns = {'household_key':'HOUSEHOLDS'}) \
            .reset_index()

fig = px.scatter(sales_agg, 
                 x='QUANTITY', 
                 y='SALES_VALUE',
                 size='HOUSEHOLDS', 
                 color='COMMODITY_DESC',
                 hover_name='COMMODITY_DESC',
                 size_max=60,
                 title = 'Valor de Venda, Quantidade e Clientes por Categoria',
                 template = 'simple_white')
fig.update_layout(showlegend=False)
fig.add_annotation(text='Os tamanhos das bolhas representam o número de famílias que compram o produto',
                  xref='paper', yref='paper',
                  x=-0.02, y=1.11, showarrow=False)
fig.show()


SOFT DRINKS se distanciam das demais categorias em quantidade, embora estejam no mesmo nível da categoria BEEF em termos de valor de venda. 

Conclue-se que SOFT DRINKS são altamente direcionados ao volume, vendem bem por causa do grande número de compras, enquanto a categoria BEEF é obviamente uma mercadoria orientada pelo preço.

Com um tempo maior de análise seria possível utilizar o algoritmo Apriori, e ver se faz sentido, por exemplo, ter produtos de SOFT DRINKS e BEEF próximos um do outro no mercado, para ajudar nas vendas explorando a ideia de produtos que podem ser comprados juntos, e descobrir outras associações.